In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
output_path = PATH['cls_train_cube_30']
output_true = PATH['cls_train_cube_30_true']
output_false = PATH['cls_train_cube_30_false']
model_paths = PATH['model_paths']
model_final = PATH['model_final']

In [3]:

def get_dirfiles(dir):
    file_list = []
    subset_path = os.listdir(dir)
    for _ in range(len(subset_path)):
        if subset_path[_] != '.DS_Store':
            file_list.append(dir + subset_path[_])
    return file_list

def train_generator(output_true,output_false):
    file_list_true = get_dirfiles(output_true)[:20]
    file_list_false = get_dirfiles(output_false)[:20]
    
    #file_list_true = shuffle(file_list_true)
    #file_list_false = shuffle(file_list_false)
    
    nb_true = len(file_list_true) + len(file_list_false)
    sample = np.zeros([nb_true,36,36,36])
    labels = np.zeros([nb_true,2])
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels

def valid_generator(output_true,output_false):
    file_list_true = get_dirfiles(output_true)[-5:]
    file_list_false = get_dirfiles(output_false)[-5:]
    
    #file_list_true = shuffle(file_list_true)
    #file_list_false = shuffle(file_list_false)

    nb_true = len(file_list_true) + len(file_list_false)
    sample = np.zeros([nb_true,36,36,36])
    labels = np.zeros([nb_true,2])
  
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels

def classifier(input_shape, width, kernel_size, pool_size):
    model = Sequential()

    model.add(Convolution3D(width*1, kernel_size[0], kernel_size[1], kernel_size[2],
                            border_mode='valid',
                            input_shape=input_shape))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Convolution3D(width*2, kernel_size[0], kernel_size[1], kernel_size[2]))
    #model.add(BatchNormalization())    
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Convolution3D(width*4, kernel_size[0], kernel_size[1], kernel_size[2]))
    #model.add(BatchNormalization())    
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
          optimizer=Adam(lr=1e-5),
          metrics=['accuracy'])
    return model


def fenlei_fit(name, load_check = False,batch_size=2, epochs=100,check_name = None):
    t = time.time()
    callbacks = [EarlyStopping(monitor='val_loss', patience = 100, verbose = 1),
                 ModelCheckpoint((model_paths + '{}.h5').format(name),
                                 monitor='val_loss',
                                 verbose = 0,
                                 save_best_only = True)]
    if load_check:
        check_model = (model_paths + '{}.h5').format(check_name)
        model = load_model(check_model)
    else:
        #model = model_20()
        #model = preds3d_dense(48)
        model = classifier((1, 36, 36, 36), 16, (3, 3, 3), (2, 2, 2))
    x,y = train_generator(output_true,output_false)
    model.fit(x=x, y=y, batch_size=batch_size, epochs=epochs,
              validation_data=valid_generator(output_true,output_false),verbose=1, callbacks=callbacks, shuffle=False) 
    return model


In [4]:
fenlei_fit('Fenge_36_36_36_0619_2', load_check = False, batch_size=2, epochs=100, check_name = 'Fenge_36_36_36_0619_2')

100%|██████████| 5/5 [00:00<00:00, 2374.22it/s]


Train on 40 samples, validate on 10 samples
Epoch 1/100
40/40 [==============================] - 2s - loss: 0.7507 - acc: 0.5500 - val_loss: 0.7286 - val_acc: 0.3000
Epoch 2/100
40/40 [==============================] - 0s - loss: 0.7102 - acc: 0.5250 - val_loss: 0.7271 - val_acc: 0.3000
Epoch 3/100
40/40 [==============================] - 0s - loss: 0.7437 - acc: 0.5500 - val_loss: 0.7267 - val_acc: 0.4000
Epoch 4/100
40/40 [==============================] - 0s - loss: 0.7379 - acc: 0.5750 - val_loss: 0.7245 - val_acc: 0.3000
Epoch 5/100
40/40 [==============================] - 0s - loss: 0.6927 - acc: 0.5750 - val_loss: 0.7268 - val_acc: 0.3000
Epoch 6/100
40/40 [==============================] - 0s - loss: 0.7760 - acc: 0.5250 - val_loss: 0.7253 - val_acc: 0.4000
Epoch 7/100
40/40 [==============================] - 0s - loss: 0.7448 - acc: 0.5250 - val_loss: 0.7256 - val_acc: 0.4000
Epoch 8/100
40/40 [==============================] - 0s - loss: 0.6145 - acc: 0.6500 - val_loss: 0.726

40/40 [==============================] - 0s - loss: 0.5026 - acc: 0.8000 - val_loss: 0.9690 - val_acc: 0.2000
Epoch 68/100
40/40 [==============================] - 0s - loss: 0.4360 - acc: 0.9000 - val_loss: 0.9736 - val_acc: 0.2000
Epoch 69/100
40/40 [==============================] - 0s - loss: 0.4885 - acc: 0.8000 - val_loss: 0.9792 - val_acc: 0.2000
Epoch 70/100
40/40 [==============================] - 0s - loss: 0.5238 - acc: 0.7750 - val_loss: 0.9831 - val_acc: 0.2000
Epoch 71/100
40/40 [==============================] - 0s - loss: 0.4947 - acc: 0.7750 - val_loss: 0.9911 - val_acc: 0.2000
Epoch 72/100
40/40 [==============================] - 0s - loss: 0.4884 - acc: 0.7750 - val_loss: 0.9959 - val_acc: 0.2000
Epoch 73/100
40/40 [==============================] - 0s - loss: 0.4026 - acc: 0.9000 - val_loss: 1.0011 - val_acc: 0.2000
Epoch 74/100
40/40 [==============================] - 0s - loss: 0.4756 - acc: 0.8500 - val_loss: 1.0106 - val_acc: 0.2000
Epoch 75/100


KeyboardInterrupt: 

In [ ]:
file_list_true = get_dirfiles(output_true)
file_list_false = get_dirfiles(output_false)

In [ ]:
#model_pred = classifier((1, 36, 36, 36), (3, 3, 3), (2, 2, 2))
model_pred = load_model(model_paths + 'Fenge_36_36_36_0619_2.h5')

In [ ]:
#model_pred.load_weights(model_paths + 'Fenge_0613_2.h5')

In [ ]:
cc = []
for i in file_list_false[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0
for i in cc:
    if i[0][0] > 0.9:
        count += 1
print count*1.0/len(cc)

In [ ]:
cc

In [ ]:
cc = []
for i in file_list_true[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0


In [ ]:
for i in cc:
    if i[0][1] > 0.9:
        count += 1
print count*1.0/len(cc)

In [ ]:
cc